In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import ast
import openai
import time
import seaborn as sns
import matplotlib.pyplot as plt
import json 
import fitz  # PyMuPDF
import pandas as pd
import glob
import spacy
from transformers import AutoModelForCausalLM, AutoTokenizer
from openai import ChatCompletion
client = ChatCompletion()

In [2]:
# Configura tu API key de OpenAI
file = open("NLP_KEY_GPT.txt", "r")
content = file.read()
file.close()

openai.api_key = content

In [3]:

pdf_resumes = []
pdf_files = glob.glob("*.pdf")  # Get a list of all PDF files in the current folder

for file in pdf_files:
    doc = fitz.open(file)
    text = ""
    for page in doc:
        text += page.get_text()
    
    pdf_resumes.append(text)
    doc.close()
pdf_resumes


[' \nPRESENTATION \n \nPlease allow me to introduce myself.  \n \nMy name is George, married to a wonderful wife who has given me two awesome \nkids, a boy and girl. I´m a Canadian – Chilean citizen, living in Santiago, Chile for the \nlast 20 years, who has also lived in Antofagasta as in other countries and cities in \nCentral and South America in the last 25 years.   \n \nI consider myself to be a resourceful professional, with 15+ years of ground \nexperience, in exceeding expectations within commercial, service, mining and \nmachinery related markets. I make every effort to be a highly effective-oriented team \nplayer who leverages research, continuous learning, adaptation and problem-solving \ndynamics to deliver timely solutions thought leadership and caring guidance to \ncustomers and employees, while creating and maintaining a trusted advisor \nrelationship. \n \nI am passionate in my approach to customer success which has enabled me to excel \nin my previous roles and no doub

In [4]:
#Resumes
def preprocess_cv(cv):
    completion = client.create(
        model="gpt-3.5-turbo",
        temperature=0.2,#to receive more similar answers
        seed=123,
        messages=[
            {"role": "system", 
             "content": '''You are a helpful and expert HR recruiter assistant, with experience to analyze resumes.
             Provide your answer in JSON strcuture like this
             {"name":"name",
             "email":"email",
             "skills":[<list of skills>],
             "education":[<list of education>],
             "experience":"2 lines summary proffesional experience",
             "years_of_experience":"<Approximation of years of experience, it should be a range 0-3,4-7,8-10,10-15,+15>"}'''},
            {"role": "user", 
             "content":
                 f'''give me a list with the name, email, the exact 10 most relevant skills based on the entire resume, 
                 education, summary of professional experience, and
                 years of experience of the following text:{cv}'''},
        ]
    )
    return eval(completion.choices[0].message.content)

def preprocess_jd(jd):
    completion = client.create(
        model="gpt-3.5-turbo",
        temperature=0.2,#to receive more similar answers
        seed=123,
        messages=[
            {"role": "system", 
             "content": '''You are a helpful and expert HR recruiter assistant, with experience to analyze resumes.
             Provide your answer in JSON strcuture like this
             {"job title":"job title",
             "role":"summary of the role",
             "skills":[<list of skills that are required for this role>],
             "education":[<list of education requirements>],
             '''},
            {"role": "user", 
             "content":
                 f'''give me the title of the position, a summary of the role, the exact 10 most relevant skills based on the entire job description, and 
                 education requirements of:{jd}'''},
        ]
    )
    return eval(completion.choices[0].message.content)


In [5]:
resumes = pd.DataFrame(columns=["name", "email", "skills", "education", "experience", "years_of_experience"])
for i in range(len(pdf_resumes)):     
    # Convert the dictionary object to a DataFrame
    new_resume = pd.DataFrame([preprocess_cv(pdf_resumes[i])])
    # Concatenate the new resume data to the DataFrame
    resumes = pd.concat([resumes, new_resume], ignore_index=True)
    # Print the DataFrame to verify the addition
#print(resumes)


In [7]:
resumes['ID'] = resumes.index
df_resumes=resumes

In [8]:
df_resumes

,name,email,skills,education,experience,years_of_experience,ID
0,George Sierra Corvalán,gscorvalan1975@gmail.com,"[Regional Security Management, Crisis Manageme...","[Safety and Industrial Risk Engineer, Diploma ...",Experienced Regional Security Manager with 15+...,10-15,0
1,JULIÁN GAONA GONZÁLEZ,julian.gaona.gonzalez@gmail.com,"[Data-driven Marketing, Commercial Strategy, B...",[MBD: Master's in Business Analytics & Big Dat...,Industrial engineering professional with over ...,10-15,1
2,John Adams,johnadams@email.com,"[Clinical Assessments, Patient Care Planning, ...","[Bachelor of Science in Nursing, [University N...",Dedicated and compassionate Nurse with over 10...,10-15,2
3,Marina Dufour Herrero,marina.dufour.herrero@gmail.com,"[Project Management, HR: Training management, ...",[Online Program Data Science and Visualization...,Marina has extensive experience in various rol...,8-10,3
4,Felipe Saavedra,felisaav@gmail.com,"[SQL, Power BI/Tableau, Python/R, Office, Visu...",[Master in Business Analytics and Big Data - I...,"Transformational leader & innovation “trader”,...",+15,4
5,Sue Smith,suesmith@email.com,"[Data Analysis, SQL and Python, Database Solut...","[Bachelor of Science in Chemistry, Ohio Univer...",Dynamic and results-oriented Senior HR Data An...,4-7,5
6,Victor Vargas Rojas,vvargasr0788@gmail.com,"[Financial Risk Management, Policy Implementat...",[{'Institution': 'IE School of Science and Tec...,Professional with more than 12 years of experi...,10-15,6


In [11]:
doc1 = fitz.open("Data Analyst.pdf")
job_desc = []
text = ""
for page in doc1:
    text += page.get_text()
doc1.close()
job_desc.append(text)  

In [12]:

df_job = pd.DataFrame(columns=["job title", "role", "skills", "education"])
for i in range(len(job_desc)):
           
    # Convert the dictionary object to a DataFrame
    new_job_desc = pd.DataFrame([preprocess_jd(job_desc[i])])
    # Concatenate the new resume data to the DataFrame
    df_job = pd.concat([df_job, new_job_desc], ignore_index=True)
    # Print the DataFrame to verify the addition
#print(resumes)


In [13]:
df_job['ID'] = df_job.index
df_job

,job title,role,skills,education,ID
0,Data Analyst - Lansweeper,The Data Analyst at Lansweeper will work as pa...,"[Analytical mindset, Proficient in SQL, Experi...",[Bachelor's degree in a related field such as ...,0


In [14]:
def semantic_similarity_sbert_base_v2(job,resume):
    """calculate similarity with SBERT all-mpnet-base-v2"""
    model = SentenceTransformer('all-mpnet-base-v2')
    #Encoding:
    score = 0
    sen = job+resume
    sen_embeddings = model.encode(sen)
    for i in range(len(job)):
        if job[i] in resume:
            score += 1
        else:
            if max(cosine_similarity([sen_embeddings[i]],sen_embeddings[len(job):])[0]) >= 0.4:
                score += max(cosine_similarity([sen_embeddings[i]],sen_embeddings[len(job):])[0])
    score = score/len(job)  
    return round(score,3)

In [15]:
def semantic_similarity_sbert_paraphrase_minilm_l6_v2(job,resume):
    """calculate similarity with SBERT paraphrase-MiniLM-L6-v2"""
    model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
    #Encoding:
    score = 0
    sen = job+resume
    sen_embeddings = model.encode(sen)
    for i in range(len(job)):
        if job[i] in resume:
            score += 1
        else:
            if max(cosine_similarity([sen_embeddings[i]],sen_embeddings[len(job):])[0]) >= 0.4:
                score += max(cosine_similarity([sen_embeddings[i]],sen_embeddings[len(job):])[0])
    score = score/len(job)  
    return round(score,3)

In [16]:
def semantic_similarity_all_MiniLM_L12_v1(job,resume):
    """calculate similarity with all-MiniLM-L12-v1"""
    model = SentenceTransformer('all-MiniLM-L12-v1')
    #Encoding:
    score = 0
    sen = job+resume
    sen_embeddings = model.encode(sen)
    for i in range(len(job)):
        if job[i] in resume:
            score += 1
        else:
            if max(cosine_similarity([sen_embeddings[i]],sen_embeddings[len(job):])[0]) >= 0.4:
                score += max(cosine_similarity([sen_embeddings[i]],sen_embeddings[len(job):])[0])
    score = score/len(job)  
    return round(score,3)

In [17]:
def semantic_similarity_multi_qa_mpnet_base_dot_v1(job,resume):
    """calculate similarity with multi-qa-mpnet-base-dot-v1"""
    model = SentenceTransformer('multi-qa-mpnet-base-dot-v1')
    #Encoding:
    score = 0
    sen = job+resume
    sen_embeddings = model.encode(sen)
    for i in range(len(job)):
        if job[i] in resume:
            score += 1
        else:
            if max(cosine_similarity([sen_embeddings[i]],sen_embeddings[len(job):])[0]) >= 0.4:
                score += max(cosine_similarity([sen_embeddings[i]],sen_embeddings[len(job):])[0])
    score = score/len(job)  
    return round(score,3)

In [18]:
def semantic_similarity_all_roberta_large_v1(job,resume):
    """calculate similarity with all-roberta-large-v1"""
    model = SentenceTransformer('all-roberta-large-v1')
    #Encoding:
    score = 0
    sen = job+resume
    sen_embeddings = model.encode(sen)
    for i in range(len(job)):
        if job[i] in resume:
            score += 1
        else:
            if max(cosine_similarity([sen_embeddings[i]],sen_embeddings[len(job):])[0]) >= 0.4:
                score += max(cosine_similarity([sen_embeddings[i]],sen_embeddings[len(job):])[0])
    score = score/len(job)  
    return round(score,3)

In [19]:
def max_similarities(skill,resume):
    similarities = []
    for j in resume:
        response_skill1 = openai.Embedding.create(
        input=skill,
        engine="text-embedding-ada-002")
        response_skill2 = openai.Embedding.create(
        input=j,
        engine="text-embedding-ada-002")
        sim = round(cosine_similarity([response_skill1["data"][0]["embedding"]],[response_skill2["data"][0]["embedding"]])[0][0], 3)
        similarities.append(sim)
        return max(similarities)
    

def semantic_similarity_gpt3(job,resume):
    """calculate similarity with GPT3"""
    score = 0
    for i in range(len(job)):
        if job[i] in resume:
            score += 1
        else:
            time.sleep(4)
            score += max_similarities(job[i],resume)
    score = score/len(job)  
    return round(score,3)

In [20]:
def max_similarities_small(skill,resume):
    similarities = []
    for j in resume:
        response_skill1 = openai.Embedding.create(
        input=skill,
        engine="text-embedding-3-small")
        response_skill2 = openai.Embedding.create(
        input=j,
        engine="text-embedding-3-small")
        sim = round(cosine_similarity([response_skill1["data"][0]["embedding"]],[response_skill2["data"][0]["embedding"]])[0][0], 3)
        similarities.append(sim)
        return max(similarities)
    

def semantic_similarity_gpt3_small(job,resume):
    """calculate similarity with GPT3 small model"""
    score = 0
    for i in range(len(job)):
        if job[i] in resume:
            score += 1
        else:
            time.sleep(4)
            score += max_similarities_small(job[i],resume)
    score = score/len(job)  
    return round(score,3)

In [21]:
def max_similarities_large(skill,resume):
    similarities = []
    for j in resume:
        response_skill1 = openai.Embedding.create(
        input=skill,
        engine="text-embedding-3-large")
        response_skill2 = openai.Embedding.create(
        input=j,
        engine="text-embedding-3-large")
        sim = round(cosine_similarity([response_skill1["data"][0]["embedding"]],[response_skill2["data"][0]["embedding"]])[0][0], 3)
        similarities.append(sim)
        return max(similarities)
    

def semantic_similarity_gpt3_large(job,resume):
    """calculate similarity with GPT3 small model"""
    score = 0
    for i in range(len(job)):
        if job[i] in resume:
            score += 1
        else:
            time.sleep(4)
            score += max_similarities_large(job[i],resume)
    score = score/len(job)  
    return round(score,3)

In [22]:
columns=['resume_id', 'job_id', 'all-mpnet-base-v2_score', 'paraphrase-MiniLM-L6-v2_score','all-MiniLM-L12-v1_score','bert_base_nli_mean_tokens_score']
matching_dataframe = pd.DataFrame(columns=columns)
ranking_dataframe = pd.DataFrame(columns=columns)

In [36]:
for index in df_job.iterrows():
    job_index = index[0]
    columns=['resume_id', 'job_id',"name", 'all-mpnet-base-v2_score', 'paraphrase-MiniLM-L6-v2_score','all-MiniLM-L12-v1_score','gpt3_score_large','gpt3_score_small']
    matching_dataframe_one_job = pd.DataFrame(columns=columns)
    ranking_dataframe_one_job = pd.DataFrame(columns=columns)




    for index in df_resumes.iterrows():
        resume_index = index[0]
        matching_dataframe_job = {}
        matching_dataframe_job = {}
        matching_dataframe_job["resume_id"] = resume_index
        matching_dataframe_job["job_id"] = df_job['ID'][job_index]
        matching_dataframe_job["name"] = df_resumes['name'][resume_index]
        matching_dataframe_job["all-mpnet-base-v2_score"] = semantic_similarity_sbert_base_v2(df_job['skills'][job_index],df_resumes['skills'][resume_index])
        matching_dataframe_job["paraphrase-MiniLM-L6-v2_score"] = semantic_similarity_sbert_paraphrase_minilm_l6_v2(df_job['skills'][job_index],df_resumes['skills'][resume_index]) 
        matching_dataframe_job["all-MiniLM-L12-v1_score"] = semantic_similarity_all_MiniLM_L12_v1(df_job['skills'][job_index],df_resumes['skills'][resume_index])
        #matching_dataframe_job["all-roberta-large-v1_score"] = semantic_similarity_all_roberta_large_v1(df_job['Skills'][job_index],df_resumes['Skills'][resume_id])
        #matching_dataframe_job["gpt3_score"] = semantic_similarity_gpt3(df_job['skills'][job_index],resumes['skills'][resume_index])
        matching_dataframe_job["gpt3_score_small"] = semantic_similarity_gpt3_small(df_job['skills'][job_index],resumes['skills'][resume_index])
        matching_dataframe_job["gpt3_score_large"] = semantic_similarity_gpt3_large(df_job['skills'][job_index],resumes['skills'][resume_index])
        #matching_dataframe_job["multi_qa_mpnet_base_dot_v1_score"] = semantic_similarity_multi_qa_mpnet_base_dot_v1(df_job['skills'][job_index],df_resumes['skills'][resume_index])
        matching_dataframe = pd.concat([matching_dataframe, pd.DataFrame(matching_dataframe_job, index=[job_index])], ignore_index=True)
        matching_dataframe_one_job = pd.concat([matching_dataframe_one_job, pd.DataFrame(matching_dataframe_job, index=[job_index])], ignore_index=True)                                        
        
    ranking_dataframe_one_job["resume_id"] = matching_dataframe_one_job["resume_id"]
    ranking_dataframe_one_job["job_id"] = matching_dataframe_one_job["job_id"]
    ranking_dataframe_one_job["name"] = matching_dataframe_one_job["name"]
    ranking_dataframe_one_job['all-mpnet-base-v2_score'] = matching_dataframe_one_job['all-mpnet-base-v2_score'].rank(ascending = False).astype(int)
    ranking_dataframe_one_job['paraphrase-MiniLM-L6-v2_score'] = matching_dataframe_one_job['paraphrase-MiniLM-L6-v2_score'].rank(ascending = False).astype(int)
    ranking_dataframe_one_job['all-MiniLM-L12-v1_score'] = matching_dataframe_one_job['all-MiniLM-L12-v1_score'].rank(ascending = False).astype(int)
    #ranking_dataframe_one_job['all-roberta-large-v1_score'] = matching_dataframe_one_job['all-roberta-large-v1_score'].rank(ascending = False).astype(int)
    #ranking_dataframe_one_job['gpt3_score'] = matching_dataframe_one_job['gpt3_score'].rank(ascending = False).astype(int)
    ranking_dataframe_one_job['gpt3_score_small'] = matching_dataframe_one_job['gpt3_score_small'].rank(ascending = False).astype(int)
    ranking_dataframe_one_job['gpt3_score_large'] = matching_dataframe_one_job['gpt3_score_large'].rank(ascending = False).astype(int)
    #ranking_dataframe_one_job['multi_qa_mpnet_base_dot_v1_score'] = matching_dataframe_one_job['multi_qa_mpnet_base_dot_v1_score'].rank(ascending = False).astype(int)
    ranking_dataframe = pd.concat([ranking_dataframe, ranking_dataframe_one_job], ignore_index=True)


C:\Users\julia\AppData\Local\Temp\ipykernel_30056\987302923.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  matching_dataframe_one_job = pd.concat([matching_dataframe_one_job, pd.DataFrame(matching_dataframe_job, index=[job_index])], ignore_index=True)


In [43]:
matching_dataframe_one_job.sort_values(by='gpt3_score_small', ascending=False)

,resume_id,job_id,name,all-mpnet-base-v2_score,paraphrase-MiniLM-L6-v2_score,all-MiniLM-L12-v1_score,gpt3_score_large,gpt3_score_small
5,5,0,Sue Smith,0.437,0.454,0.428,0.401,0.393
1,1,0,JULIÁN GAONA GONZÁLEZ,0.299,0.381,0.266,0.280,0.314
4,4,0,Felipe Saavedra,0.362,0.383,0.289,0.256,0.293
3,3,0,Marina Dufour Herrero,0.233,0.387,0.232,0.299,0.267
2,2,0,John Adams,0.043,0.055,0.000,0.227,0.249
6,6,0,Victor Vargas Rojas,0.179,0.197,0.168,0.212,0.218
0,0,0,George Sierra Corvalán,0.042,0.041,0.041,0.194,0.181


In [38]:
ranking_dataframe_one_job.sort_values(by='gpt3_score_small', ascending=True)



,resume_id,job_id,name,all-mpnet-base-v2_score,paraphrase-MiniLM-L6-v2_score,all-MiniLM-L12-v1_score,gpt3_score_large,gpt3_score_small
5,5,0,Sue Smith,1,1,1,1,1
1,1,0,JULIÁN GAONA GONZÁLEZ,3,4,3,3,2
4,4,0,Felipe Saavedra,2,3,2,4,3
3,3,0,Marina Dufour Herrero,4,2,4,2,4
2,2,0,John Adams,6,6,7,5,5
6,6,0,Victor Vargas Rojas,5,5,5,6,6
0,0,0,George Sierra Corvalán,7,7,6,7,7
